# 6. SED fitting to spatial bins

<font size="4"> In this step, we will perform SED fitting to each spatial bin in the binned spectrophotometric data cube we obtained from the Step 5. This SED fitting can be performed using functions in the `piXedfit_fitting` module. We will also learn how to check (i.e., analyze) fitting results using visualization features provided in the `piXedfit_analysis` module. </font>

In [1]:
import numpy as np
from astropy.io import fits
import os, sys
import matplotlib.pyplot as plt

global PIXEDFIT_HOME
PIXEDFIT_HOME = os.environ['PIXEDFIT_HOME']
sys.path.insert(0, PIXEDFIT_HOME)

%matplotlib inline

<font size="4"> There are several functions available in the `piXedfit_fitting` module that can be used for performing SED fitting, depending on the type of data we have. If we have a single SED or a collection of them, but we want to fit the SEDs one by one, we can use `singleSEDfit` function (for photometric SED) or `singleSEDfit_specphoto` (for spectrophotometric SED). If we have a 3D data cube obtained after image processing and pixel binning, we can use `SEDfit_from_binmap` (for photometric data cube) or `SEDfit_from_binmap_specphoto` (for spectrphotmetric data cube). We can also perform SED fitting to individual pixels in the data cube after image processing (before pixel binning) using `SEDfit_pixels_from_fluxmap` function. Please refer to the API references for more information about these functions. 
    
Since we have a spectrophotometric data cube in this tutorial, we will use `SEDfit_from_binmap_specphoto` for fitting the SEDs of individual spatial bins. An example of script for doing this is given below. In this script, `binid_range` determines the spatial bins that are going to be fit. With this, we can separate calculations into multiple terminals (i.e., processes) on a multi cores server or super computer.  
    
`models_spec` is a HDF5 file containing a set of model spectra at rest-frame that is generated using `piXedfit.piXedfit_model.save_models_rest_spec` function. Please refer to  
</font>

In [ ]:
# Script 1
from piXedfit.piXedfit_fitting import SEDfit_from_binmap_specphoto

fits_binmap = "pixbin_corr_specphoto_fluxmap_ngc309.fits"
binid_range = [0,10]
models_spec = "model_rest_spec.hdf5"     # set of model rest-frame spectra  

# define priors using function priors
from piXedfit.piXedfit_fitting import priors

# define range of values for some parameters   
ranges = {'logzsol':[-2.0,0.2], 'dust1':[0.0,3.0], 'dust2':[0.0,3.0], 'log_age':[-3.0,1.14]}
pr = priors(ranges)
params_ranges = pr.params_ranges()

prior1 = pr.uniform('logzsol')
prior2 = pr.uniform('dust1')
prior3 = pr.uniform('dust2')
prior4 = pr.uniform('log_age')
params_priors = [prior1, prior2, prior3, prior4]

fit_method = 'mcmc'

SEDfit_from_binmap_specphoto(fits_binmap, binid_range=binid_range, models_spec=models_spec,
                             params_ranges=params_ranges, params_priors=params_priors,
                             fit_method=fit_method, spec_sigma=2.6, poly_order=10, nwalkers=100,
                             nsteps=700, nproc=10, initfit_nmodels_mcmc=50000, cosmo=0, H0=70.0, Om0=0.3,
                             store_full_samplers=1)